In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import imblearn
from os import path
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import sklearn.preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
import src.lib.utility_classfier as uclf
import src.lib.optimal_threhold_related as thres
import src.lib.fairness_tests as fair

In [2]:
data_path='/Users/lifuchen/Desktop/research/data.csv'
df = pd.read_csv(data_path)
df.shape

(109490, 89)

In [3]:
y = df.Class.values
X = df.drop(['GRID','Class'], axis=1)
X.shape

(109490, 87)

In [4]:
def get_result (classifier, records, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female):
    method_to_call = getattr(uclf, classifier)
    y_val_score = method_to_call(X_train_scaled, y_train,X_val_scaled, y_val)
    y_test_score = method_to_call(X_train_scaled, y_train,X_test_scaled, y_test)

    threshold, ba_val, ba_test = balance_accuracy (y_val, y_val_score,y_test, y_test_score)
    auroc = roc_auc_score(y_test, y_test_score)
    precision, recall, tpr, tnr, pd = thres.calculate_precision_metrics(y_test, y_test_score,threshold)

    y_val_score_male = method_to_call(X_train_scaled, y_train, X_val_male_scaled, y_val_male)
    y_test_score_male = method_to_call(X_train_scaled, y_train,X_test_male_scaled, y_test_male)
    threshold_male, ba_val_male, ba_test_male = balance_accuracy (y_val_male, y_val_score_male,y_test_male, y_test_score_male)
    precision_male, recall_male, tpr_male, tnr_male, pd_male = thres.calculate_precision_metrics(y_test_male, y_test_score_male,threshold_male)

    y_val_score_female = method_to_call(X_train_scaled, y_train, X_val_female_scaled, y_val_female)
    y_test_score_female = method_to_call(X_train_scaled, y_train,X_test_female_scaled, y_test_female)
    threshold_female, ba_val_female, ba_test_female = balance_accuracy (y_val_female, y_val_score_female, y_test_female, y_test_score_female)
    precision_female, recall_female, tpr_female, tnr_female, pd_female = thres.calculate_precision_metrics(y_test_female, y_test_score_female,threshold_female)

    eod = fair.get_EOD(y_test_male, y_test_score_male,threshold_male, y_test_female, y_test_score_female, threshold_female)
    sp = fair.get_SP(y_test_male, y_test_score_male,threshold_male, y_test_female, y_test_score_female, threshold_female)

    records.append({
        'auroc': auroc,
        'overall threshold': threshold,
        'male threshold': threshold_male,
        'female threshold': threshold_female,
        'overall ba validation': ba_val,
        'overall ba test': ba_test,
        'male ba validation': ba_val_male,
        'male ba test': ba_test_male,
        'female ba validation': ba_val_female,
        'female ba test': ba_test_female,
        'overall precision':precision,
        'overall recall':recall,
        'overall tpr':tpr,
        'overall tnr':tnr,
        'overall pd':pd,
        'male precision':precision_male,
        'male recall':recall_male,
        'male tpr':tpr_male,
        'male tnr':tnr_male,
        'male pd':pd_male,
        'female precision':precision_female,
        'female recall':recall_female,
        'female tpr':tpr_female,
        'female tnr':tnr_female,
        'female pd':pd_female,
        'eod': eod,
        'di': sp,
        })

In [5]:
def balance_accuracy (y_val, y_val_score,y_test, y_test_score):
    
    threshold, _ = thres.get_optimal_threshold_Jvalue (y_val, y_val_score)
    print ("Optimal threshold by J value is ",threshold)

    ba_val = thres.calculate_balanced_accuracy(y_val, y_val_score, threshold)
    print ("Balanced accuracy score of val is ", ba_val)

    ba_test = thres.calculate_balanced_accuracy(y_test, y_test_score, threshold)
    print ("Balanced accuracy score of test is ",ba_test)

    return threshold, ba_val, ba_test

In [6]:
def fairness_metrics (X, y, attribute, random_state):
    X_train, y_train, X_val, y_val, X_test, y_test, X_val_female, X_val_male, y_val_female, y_val_male, X_test_female, X_test_male, y_test_female, y_test_male \
        = fair.split_by_trait_balance_size(X, y, attribute, random_state)
    
    print("X train", X_train.shape[0])
    print("Y train", y_train.shape[0])
    print(X_val.shape[0], X_val_male.shape[0], X_val_female.shape[0])
    print(y_val.shape[0], y_val_male.shape[0], y_val_female.shape[0])
    print(X_test.shape[0], X_test_male.shape[0], X_test_female.shape[0])
    print(y_test.shape[0], y_test_male.shape[0], y_test_female.shape[0])

    max_abs_scaler = preprocessing.MaxAbsScaler()
    X_train_scaled = max_abs_scaler.fit_transform(X_train)
    X_test_scaled = max_abs_scaler.transform(X_test)
    X_test_male_scaled = max_abs_scaler.transform(X_test_male)
    X_test_female_scaled = max_abs_scaler.transform(X_test_female)
    X_val_scaled = max_abs_scaler.transform(X_val)
    X_val_male_scaled = max_abs_scaler.transform(X_val_male)
    X_val_female_scaled = max_abs_scaler.transform(X_val_female)

    get_result ("logic_regression", records_lr, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)
    get_result ("random_forest", records_rf, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)
    get_result ("decision_tree", records_dt, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)
    get_result ("gradiant_boosting", records_gbt, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)

In [ ]:
records_lr = []
records_rf = []
records_dt = []
records_gbt = []
for random_state in range(0,2):
    fairness_metrics (X, y, "GENDER", random_state)

result_lr = pd.DataFrame(records_lr)
result_rf = pd.DataFrame(records_rf)
result_dt = pd.DataFrame(records_dt)
result_gbt = pd.DataFrame(records_gbt)

/Users/lifuchen/Desktop/research/CVDPrediction-master/src/lib/fairness_tests.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train ['Class'] = y_train


(42330,)
(42330,)
(84660, 87)
X train 84660
Y train 84660
21898 7782 14116
21898 7782 14116
21898 7707 14191
21898 7707 14191


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.22069183751550578
0.26394338912829324
Classification report
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     19904
           1       0.44      0.04      0.08      1994

    accuracy                           0.91     21898
   macro avg       0.67      0.52      0.51     21898
weighted avg       0.87      0.91      0.87     21898

Confusion_matrix
[[19797   107]
 [ 1911    83]]
done in 1.140168s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

0.22069183751550578
0.3045448413287432
Classification report
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      6860
           1       0.50      0.04      0.07       847

    accuracy                           0.89      7707
   macro avg       0.70      0.52      0.50      7707
weighted avg       0.85      0.89      0.85      7707

Confusion_matrix
[[6830   30]
 [ 817   30]]
done in 0.807960s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.359
threshold:0.2, J-value:0.21799999999999997
threshold:0.30000000000000004, J-value:0.11699999999999999
threshold:0.4, J-value:0.05
threshold:0.5, J-value:0.027000000000000003
threshold:0.6000000000000001, J-value:0.009000000000000001
threshold:0.7000000000000001, J-value:0.002
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.6792685276228649
Balanced accuracy score of test is  0.6800050598751898


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.22069183751550578
0.23853684977256281
Classification report
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     12989
           1       0.45      0.05      0.09      1127

    accuracy                           0.92     14116
   macro avg       0.69      0.52      0.52     14116
weighted avg       0.89      0.92      0.89     14116

Confusion_matrix
[[12924    65]
 [ 1073    54]]
done in 0.803638s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.22069183751550578
0.24103427965193772
Classification report
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     13074
           1       0.39      0.05      0.08      1117

    accuracy                           0.92     14191
   macro avg       0.66      0.52      0.52     14191
weighted avg       0.88      0.92      0.89     14191

Confusion_matrix
[[12991    83]
 [ 1064    53]]
done in 0.815861s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.41600000000000004
threshold:0.2, J-value:0.263
threshold:0.30000000000000004, J-value:0.159
threshold:0.4, J-value:0.097
threshold:0.5, J-value:0.043000000000000003
threshold:0.6000000000000001, J-value:0.019000000000000003
threshold:0.7000000000000001, J-value:0.005
threshold:0.8, J-value:0.001
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.708223387163379
Balanced accuracy score of test is  0.6871215417397477
True positive rate of cl

In [ ]:
result_lr

In [ ]:
def add_mean_sd(records, result_table, overall_records, type):
    records.append({
        'auroc': result_table["auroc"].mean(),
        'overall threshold': result_table["overall threshold"].mean(),
        'male threshold': result_table["male threshold"].mean(),
        'female threshold': result_table["female threshold"].mean(),
        'overall ba validation': result_table["overall ba validation"].mean(),
        'overall ba test': result_table["overall ba test"].mean(),
        'male ba validation': result_table["male ba validation"].mean(),
        'male ba test': result_table["male ba test"].mean(),
        'female ba validation': result_table["female ba validation"].mean(),
        'female ba test': result_table["female ba test"].mean(),
        'overall precision':result_table["overall precision"].mean(),
        'overall recall':result_table["overall recall"].mean(),
        'overall tpr':result_table["overall tpr"].mean(),
        'overall tnr':result_table["overall tnr"].mean(),
        'overall pd':result_table["overall pd"].mean(),
        'male precision':result_table["male precision"].mean(),
        'male recall':result_table["male recall"].mean(),
        'male tpr':result_table["male tpr"].mean(),
        'male tnr':result_table["male tnr"].mean(),
        'male pd':result_table["male pd"].mean(),
        'female precision':result_table["female precision"].mean(),
        'female recall':result_table["female recall"].mean(),
        'female tpr':result_table["female tpr"].mean(),
        'female tnr':result_table["female tnr"].mean(),
        'female pd':result_table["female pd"].mean(),
        'eod': result_table["eod"].mean(),
        'di': result_table["di"].mean(),
        })
    records.append({
        'auroc': result_table["auroc"].std(),
        'overall threshold': result_table["overall threshold"].std(),
        'male threshold': result_table["male threshold"].std(),
        'female threshold': result_table["female threshold"].std(),
        'overall ba validation': result_table["overall ba validation"].std(),
        'overall ba test': result_table["overall ba test"].std(),
        'male ba validation': result_table["male ba validation"].std(),
        'male ba test': result_table["male ba test"].std(),
        'female ba validation': result_table["female ba validation"].std(),
        'female ba test': result_table["female ba test"].std(),
        'overall precision':result_table["overall precision"].std(),
        'overall recall':result_table["overall recall"].std(),
        'overall tpr':result_table["overall tpr"].std(),
        'overall tnr':result_table["overall tnr"].std(),
        'overall pd':result_table["overall pd"].std(),
        'male precision':result_table["male precision"].std(),
        'male recall':result_table["male recall"].std(),
        'male tpr':result_table["male tpr"].std(),
        'male tnr':result_table["male tnr"].std(),
        'male pd':result_table["male pd"].std(),
        'female precision':result_table["female precision"].std(),
        'female recall':result_table["female recall"].std(),
        'female tpr':result_table["female tpr"].std(),
        'female tnr':result_table["female tnr"].std(),
        'female pd':result_table["female pd"].std(),
        'eod': result_table["eod"].std(),
        'di': result_table["di"].std(),
        })
    overall_records.append({
        'type': type,
        'auroc': result_table["auroc"].mean(),
        'overall threshold': result_table["overall threshold"].mean(),
        'male threshold': result_table["male threshold"].mean(),
        'female threshold': result_table["female threshold"].mean(),
        'overall ba test': result_table["overall ba test"].mean(),
        'male ba test': result_table["male ba test"].mean(),
        'female ba test': result_table["female ba test"].mean(),
        'overall tpr':result_table["overall tpr"].mean(),
        'overall pd':result_table["overall pd"].mean(),
        'male tpr':result_table["male tpr"].mean(),
        'male pd':result_table["male pd"].mean(),
        'female tpr':result_table["female tpr"].mean(),
        'female pd':result_table["female pd"].mean(),
        'eod': result_table["eod"].mean(),
        'di': result_table["di"].mean(),
        })
    pd_result = pd.DataFrame(records)
    return pd_result, overall_records

In [ ]:
overall_table = []
result_lr, overall_records = add_mean_sd (records_lr, result_lr, overall_table, 'lr')
result_rf, overall_records = add_mean_sd (records_rf, result_rf, overall_records, 'rf')
result_dt, overall_records = add_mean_sd (records_dt, result_dt, overall_records, 'dt')
result_gbt, overall_records = add_mean_sd (records_gbt, result_gbt, overall_records, 'gbt')

result_path='/Users/lifuchen/Desktop/research/resample_data/'
result_lr.to_csv(path.join(result_path,'gender-lr-resample-size-result.csv'), index=False)
result_rf.to_csv(path.join(result_path,'gender-rf-resample-size-result.csv'), index=False)
result_dt.to_csv(path.join(result_path,'gender-dt-resample-size-result.csv'), index=False)
result_gbt.to_csv(path.join(result_path,'gender-gbt-resample-size-result.csv'), index=False)

overall_result = pd.DataFrame(overall_table)
result_path='/Users/lifuchen/Desktop/research/resample_result/'
overall_result.to_csv(path.join(result_path,'gender-resample-size.csv'), index=False)
